## Assignment 1: Line Charts

* Import the file 'spanish_skiers.csv'.
* Build a Line chart of `Percent_Skiers` vs `Year` and give it a title. 


In [2]:
import pandas as pd
import plotly.express as px

In [3]:
spain_skiers = (
    pd.read_csv("../Data/spanish_skiers.csv")
    .rename({"Percent_Skiers": "Percentage of Spaniards That Skied"}, axis=1)
)

spain_skiers.head()

Year  Percentage of Spaniards That Skied
0  2000.0                                 0.8
1  2001.0                                 0.7
2  2002.0                                 0.8
3  2003.0                                 1.0
4  2004.0                                 0.8

In [4]:
px.line(
    spain_skiers, 
    x="Year", 
    y="Percentage of Spaniards That Skied", 
    color_discrete_sequence=px.colors.qualitative.Dark2,
    title="Percent of Spanish Who Skied Each Year"
)

## Assignment 2: Bar Charts

* Use the ski_resorts dataframe below
* Create a Bar Chart of `TotalLifts` by `Country`
* Then, modify the chart to build a stacked barchart where the height of the bars are the sum of `SurfaceLifts`, `ChairLifts` and `GondolaLifts` (should equal `TotalLifts`) but the categories are visible.

In [5]:
ski_resorts = pd.read_csv("../Data/European_Ski_Resorts.csv").drop("Unnamed: 0", axis=1)

ski_resorts.head()

Resort          Country  \
0                              Alpendorf (Ski amedé)          Austria   
1  Soldeu-Pas de la Casa/​Grau Roig/​El Tarter/​C...          Andorra   
2                              Oberau (Wildschönau)          Austria   
3                                     Dachstein West          Austria   
4                                        Rosa Khutor  Southern Russia   

   HighestPoint  LowestPoint  DayPassPriceAdult  BeginnerSlope  \
0          1980          740                 52             30   
1          2640         1710                 47            100   
2          1130          900                 30              1   
3          1620          780                 42             15   
4          2320          940                 22             30   

   IntermediateSlope  DifficultSlope  TotalSlope Snowparks NightSki  \
0                 81               4         115       Yes       No   
1                 77              33         210       Yes      Yes   
2                  0               1           2        No       No   
3                 33               3          51       Yes      Yes   
4                 26              21          77       Yes       No   

   SurfaceLifts  ChairLifts  GondolaLifts  TotalLifts  LiftCapacity  \
0            22          16            11          49         75398   
1            37          28             7          72         99017   
2             2           0             0           2          1932   
3            25           8             3          36         32938   
4             6          11            10          27         49228   

   SnowCannons  
0          600  
1         1032  
2            0  
3          163  
4          450

In [6]:
lift_type_by_country = (
    ski_resorts
    .groupby("Country", as_index=False)
    .agg({"SurfaceLifts":"sum", "ChairLifts":"sum", "GondolaLifts":"sum", "TotalLifts":"sum"})
    .sort_values("TotalLifts", ascending=False)
)

lift_type_by_country.head()

Country  SurfaceLifts  ChairLifts  GondolaLifts  TotalLifts
7        France          1805        1330           448        3583
1       Austria          1238         844           431        2513
24  Switzerland           653         350           282        1285
10        Italy           446         552           206        1204
0       Andorra           136         100            26         262

In [7]:
px.bar(
    lift_type_by_country,
    x="Country",
    y="TotalLifts",
    title="Total Ski Lifts by Country"
)

In [9]:
# Solution using 'wide data' 

px.bar(
    lift_type_by_country,
    x="Country",
    y=["SurfaceLifts","ChairLifts", "GondolaLifts"],
    title="Total Ski Lifts by Country and Lift Type"
).update_layout(legend_title_text="Lift Type")

## Assignment 3: Bubble & Donut Charts

Using the `ski_resorts` DataFrame from Assignment 2:
* Build a bubble chart comparing `TotalLifts` to `LiftCapacity` with size as the number of `Gondola Lifts` for each country.
* Make Sure to color your dots by country

Then, identify the ski resort with the greatest total lift capacity:
* Build a Donut Chart with a slice for each lift type

In [11]:
# Bubble Chart

px.scatter(
    ski_resorts, 
    "TotalLifts", 
    "LiftCapacity", 
    color="Country", 
    size="GondolaLifts",
    title="Total Lifts Vs. Lift Capacity at European Resorts (Size = Gondola Lifts)"
)

In [13]:
les_gets = (
    ski_resorts
    .loc[ski_resorts["LiftCapacity"].argmax()].iloc[[-6, -5, -4]]
    .reset_index()
    .rename({"index": "lift_type", 265: "count"}, axis=1)
)

les_gets

lift_type count
0  SurfaceLifts    89
1    ChairLifts    74
2  GondolaLifts    11

In [14]:
# Donut Chart

px.pie(
    les_gets,
    values="count",
    names="lift_type",
    hole=.75,
    category_orders = {"lift_type": ["SurfaceLifts", "ChairLifts", "GondolaLifts"]},
    title="Les Gets Resort Ski Lift by Type"
)

## Assignment 4: Additional Formatting

* We will be modifying the lift type by country data we created in assigment 2. We just want the top 10 rows
* Center the title, and change the font color to a color of your choice
* Look up options for modifying the background of the plot and figure, change them to white
* Change the y-axis title to "Total Lifts"
* Because we changed the background color to white, change the grid line colors to black `gridcolor`="black"


In [15]:
lift_type_by_country = (
    ski_resorts
    .groupby("Country", as_index=False)
    .agg({"SurfaceLifts":"sum", "ChairLifts":"sum", "GondolaLifts":"sum", "TotalLifts":"sum"})
    .sort_values("TotalLifts", ascending=False)
    .iloc[:10]
)

px.bar(
    lift_type_by_country,
    x="Country",
    y="TotalLifts",
    title="Total Ski Lifts by Country"
).update_layout(
    title = {"x": .5, "y": .85, "font": {"color":"darkslateblue", "size": 30}},
    plot_bgcolor="#FFFFFF",
    paper_bgcolor="#FFFFFF"
).update_yaxes(
    {"title": "Total Lifts"},
    gridcolor="black"
)

## Assignment 5: Maps

Build a Choropleth Map of LiftCapacity by Country

* Use Location Mode = 'country names', which will allow you to specify the `Country` column in the locations argument. 
* Consider changing the scope to `europe`
* You will need to aggregate your dataframe to get a correct map

In [17]:
px.choropleth(
    (ski_resorts
     .groupby("Country", as_index=False)
     .agg({"LiftCapacity": "sum"})
    ),
    locations="Country",
    color="LiftCapacity",
    locationmode="country names",
    scope="europe",
    # may need to play with height/width/margins for ideal view
    height=800,
    width=1000
).update_layout(coloraxis_showscale=False) # may want to remove scale